# IMPORT

In [2]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, lit, col, when, count, isnan
from pyspark.errors import PySparkValueError
from requests import HTTPError
from datetime import date, timedelta
import requests
import gzip
from io import BytesIO
import json
from pyspark.sql.types import *
import matplotlib.pyplot as plt

# CONST

In [145]:
API_KEY = "fb24501dcc147d8b2a12ae4312215c55"
BASE_URL = "https://api.themoviedb.org/3"
CHANGE_URL = "movie/changes"
GET_MOVIE_URL = "movie/"

# CREATE SPARK SESSION

In [14]:
spark1 = SparkSession.builder.appName("Updater").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 07:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# RETRIEVE ALL MOVIE ID AND COMPARE TO EXISTING

In [24]:
today = date.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime("%m_%d_%Y")
yesterday_str = yesterday.strftime("%m_%d_%Y")


def retrieve_tmdb_movie_id(date_str):
    url = f"http://files.tmdb.org/p/exports/movie_ids_{date_str}.json.gz"
    response = requests.get(url)

    movie_ids = []

    if response.status_code != 200:
        print("Failed to download the file: HTTP %d", response.status_code)

    with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
        for line in f:
            movie = json.loads(line)
            movie_ids.append(movie['id'])
    return movie_ids

new_movies_ids = list(set(retrieve_tmdb_movie_id(today_str)) - set(retrieve_tmdb_movie_id(yesterday_str)))


# OPEN MOVIE CSV FILE

In [15]:
df = spark1.read.csv("./data/tmdb/load/TMDB_all_movies_today.csv", header=True, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)
df1 = spark1.read.csv("./data/tmdb/load/TMDB_all_movies_yesterday.csv", header=True, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)
df = df.sort(df.id.asc())
df1 = df.sort(df.id.asc())


In [ ]:

#df1.filter(df.id == 1465528).show()
id_list = df.select('id').rdd.flatMap(lambda x: x).collect()

436


In [26]:
new_ids_list_clean = list(set(new_movies_ids) - set(id_list))

In [27]:
print(len(new_ids_list_clean))

362


1120929

1079565


In [203]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import requests

columns = [
    "id", "title", "vote_average", "vote_count", "status", "release_dates", "revenue", "runtime",
    "budget", "imdb_id", "original_language", "original_title", "overview", "popularity",
    "tagline", "genres", "production_companies", "production_countries", "spoken_languages",
    "cast", "director", "director_of_photography", "writers", "producers", "music_composer",
    "imdb_rating", "imdb_votes", "poster_path", "modification"
]

# Clé API
movie_ids = [273125, 1005299, 232672]  # exemple de IDs
schema = df.schema
# Préparation des données
rows = []

for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/changes?api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        changes = response.json().get("changes", [])
        for change in changes:
            print(change.get("key"))
            if change.get("key") in columns:
                row = [None] * (len(schema.fields))  # créer une ligne vide
                row[0] = movie_id                    # position 0 = id
                row[-1] = change.get("key")          # dernière position = modification
                rows.append(row)
    else:
        print(f"Erreur pour le film {movie_id} : {response.status_code}")

# Création du DataFrame PySpark
df = spark1.createDataFrame(rows, schema=schema)
df.show(truncate=False)


release_dates
images
+------+-----+------------+----------+------+------------+-------+-------+------+-------+-----------------+--------------+--------+----------+-------+------+--------------------+--------------------+----------------+----+--------+-----------------------+-------+---------+--------------+-----------+----------+-------------+
|id    |title|vote_average|vote_count|status|release_date|revenue|runtime|budget|imdb_id|original_language|original_title|overview|popularity|tagline|genres|production_companies|production_countries|spoken_languages|cast|director|director_of_photography|writers|producers|music_composer|imdb_rating|imdb_votes|poster_path  |
+------+-----+------------+----------+------+------------+-------+-------+------+-------+-----------------+--------------+--------+----------+-------+------+--------------------+--------------------+----------------+----+--------+-----------------------+-------+---------+--------------+-----------+----------+-------------+
|273